In [1]:
import requests
import json
import geopandas as gpd

In [2]:
svi_file_path = "SVI.geojson"
svi_gdf = gpd.read_file(svi_file_path)

# Extract all unique FIPS codes
fips_codes = svi_gdf["FIPS"].unique()
print(f"Number of FIPS codes in SVI: {len(fips_codes)}")

Number of FIPS codes in SVI: 222


In [3]:
shapefile_path = "TIGER_2022_Tract/tl_2022_12_tract.shp"
tracts_gdf = gpd.read_file(shapefile_path)
print(f"Number of census tracts in TIGER shapefile: {len(tracts_gdf)}")

Number of census tracts in TIGER shapefile: 5160


In [4]:
# Check FIPS codes in SVI dataset
print(svi_gdf["FIPS"].head())

0    12071000303
1    12071000304
2    12071000305
3    12071000306
4    12071000401
Name: FIPS, dtype: int64


In [5]:
# Check FIPS codes in the TIGER shapefile
print(tracts_gdf["GEOID"].head())

0    12071080203
1    12071040110
2    12071040124
3    12011060303
4    12086013600
Name: GEOID, dtype: object


In [6]:
# Convert FIPS to string with leading zeros preserved
svi_gdf["FIPS"] = svi_gdf["FIPS"].astype(str)

# Check the first few FIPS codes after conversion
print(svi_gdf["FIPS"].head())

0    12071000303
1    12071000304
2    12071000305
3    12071000306
4    12071000401
Name: FIPS, dtype: object


In [7]:
# Check data types and leading zeros
print(svi_gdf["FIPS"].dtype)  # Should be object (string)
print(tracts_gdf["GEOID"].dtype)  # Should be object (string)

object
object


In [8]:
# Create a dictionary to store FIPS-to-polygon mapping
fips_to_polygon = {}

for fips in svi_gdf["FIPS"]:
    matching_row = tracts_gdf[tracts_gdf["GEOID"] == fips]
    if not matching_row.empty:
        fips_to_polygon[fips] = matching_row.iloc[0].geometry
    else:
        print(f"Polygon not found for FIPS: {fips}")

In [9]:
fips_to_polygon

{'12071000303': <POLYGON ((-81.853 26.677, -81.851 26.678, -81.85 26.679, -81.846 26.684, -8...>,
 '12071000304': <POLYGON ((-81.87 26.665, -81.867 26.667, -81.866 26.668, -81.863 26.67, -81...>,
 '12071000305': <POLYGON ((-81.857 26.652, -81.857 26.652, -81.857 26.652, -81.856 26.652, -...>,
 '12071000306': <POLYGON ((-81.829 26.667, -81.828 26.668, -81.827 26.668, -81.827 26.668, -...>,
 '12071000401': <POLYGON ((-81.817 26.677, -81.816 26.678, -81.816 26.678, -81.816 26.679, -...>,
 '12071000402': <POLYGON ((-81.8 26.67, -81.8 26.67, -81.8 26.67, -81.8 26.671, -81.799 26.6...>,
 '12071000502': <POLYGON ((-81.86 26.64, -81.86 26.641, -81.86 26.641, -81.86 26.641, -81.86...>,
 '12071000503': <POLYGON ((-81.854 26.648, -81.852 26.651, -81.852 26.651, -81.852 26.651, -...>,
 '12071000504': <POLYGON ((-81.828 26.648, -81.828 26.648, -81.828 26.648, -81.828 26.648, -...>,
 '12071000600': <POLYGON ((-81.86 26.639, -81.86 26.639, -81.86 26.639, -81.858 26.639, -81....>,
 '12071000700': <POL

In [10]:
# Function to get the geometry for a FIPS code
def get_geometry(fips):
    return fips_to_polygon.get(fips, None)  # Return None if FIPS is not found

# Add geometry to the SVI GeoDataFrame
svi_gdf["geometry"] = svi_gdf["FIPS"].apply(get_geometry)

# Check if geometries are added correctly
print(svi_gdf[["FIPS", "geometry"]].head())

          FIPS                                           geometry
0  12071000303  POLYGON ((-81.85289 26.67656, -81.85109 26.678...
1  12071000304  POLYGON ((-81.86987 26.66544, -81.86716 26.667...
2  12071000305  POLYGON ((-81.85662 26.65173, -81.85656 26.651...
3  12071000306  POLYGON ((-81.8286 26.6674, -81.82765 26.66792...
4  12071000401  POLYGON ((-81.81665 26.67722, -81.8161 26.6777...


In [11]:
# Count rows with missing geometries
missing_geometry_count = svi_gdf["geometry"].isnull().sum()
print(f"Number of rows without geometries: {missing_geometry_count}")

Number of rows without geometries: 0


In [12]:
# Save the updated GeoJSON
output_geojson_path = "Updated_SVI.geojson"
svi_gdf.to_file(output_geojson_path, driver="GeoJSON")

print(f"Updated SVI.geojson saved to: {output_geojson_path}")

Updated SVI.geojson saved to: Updated_SVI.geojson
